<a href="https://colab.research.google.com/github/andreaeusebi/pytorch_for_deep_learning/blob/main/exercises/01_pytorch_workflow_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. PyTorch Workflow Fundamentals Exercises